In [1]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.pipeline import Pipeline
from matplotlib import pyplot as plt
from datetime import datetime
from datetime import timedelta
import json
import requests
import pandas as pd
import numpy as np
import plotly.graph_objects as go
import tqdm
import warnings
import csv

warnings.simplefilter('ignore')

ModuleNotFoundError: No module named 'plotly'

## Data loading

In [ ]:
INPUT_PREFIX = './data/time_series_covid_19'

df_confirmed = pd.read_csv(
    INPUT_PREFIX + '_confirmed.csv').drop(['Province/State', 'Lat', 'Long'], axis=1).groupby(by='Country/Region').sum().reset_index()
df_recovered = pd.read_csv(INPUT_PREFIX + '_recovered.csv').drop(['Province/State', 'Lat', 'Long'], axis=1).groupby(by='Country/Region').sum().reset_index()
df_deaths = pd.read_csv(INPUT_PREFIX + '_deaths.csv').drop(['Province/State', 'Lat', 'Long'], axis=1).groupby(by='Country/Region').sum().reset_index()

## Data aggregation

In [ ]:
dates = list(df_confirmed.columns)[1::]

In [ ]:
df_merged = pd.DataFrame(columns=['country', 'date', 'confirmed', 'recovered', 'deaths'])

for r, row in tqdm.tqdm(df_confirmed.iterrows(), total=df_confirmed.shape[0]):
    for date in dates:
        df_merged = df_merged.append(
            {
                'country': row['Country/Region'],
                'date_raw': date,
                'confirmed': df_confirmed[df_confirmed['Country/Region']==row['Country/Region']][date].values[0],
                'recovered': df_recovered[df_recovered['Country/Region']==row['Country/Region']][date].values[0],
                'deaths': df_deaths[df_deaths['Country/Region']==row['Country/Region']][date].values[0]
            },
            ignore_index=True
        )

df_merged['active'] = df_merged['confirmed'] - df_merged['recovered'] - df_merged['deaths'] 

In [ ]:
df_merged['date'] = df_merged['date_raw'].map(lambda d: datetime.strptime(d, '%m/%d/%y'))

In [ ]:
df_merged = df_merged[df_merged['country'].isin(['Italy', 'China', 'Spain', 'Portugal', 'France', 'US', 'Japan', 'Brazil'])]

In [ ]:
df_merged_filtered=df_merged[df_merged['country']=='Brazil']
df_merged_filtered

In [ ]:
df_filtered = df_merged_filtered[df_merged_filtered['confirmed']>=1]

days_from_k_cases = []

for _, row in df_filtered.iterrows():
    aux = (row['date'] - df_filtered[df_filtered['country'] == row['country']]['date'].min()).days + 1
    days_from_k_cases.append(aux)
    
df_filtered['days_from_k_case'] = days_from_k_cases

initial_day = df_filtered[df_filtered['country'] == row['country']]['date'].min()

## Grouping country for regression modelling

In [ ]:
X_raw, y_raw = [], []

for country, df_country in df_filtered.groupby(by='country'):
    X_raw += list(df_country['days_from_k_case'].values)
    y_raw += list(df_country['active'].values)
    
X = np.array(X_raw).reshape(-1, 1)
y = np.array(y_raw)



i = 0
writer = csv.writer(open("covid19.csv", 'w'), delimiter=',')
for row in y:
    writer.writerow((X[i][0], y[i]))
    i = i + 1

In [ ]:
# Split the whole set into random training and validation set.
x_train, x_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=10)

include_bias = False
error_train = []
error_val = []
mean_vec = []
std_vec = []
for d in range(1, 13):
    # Instantiate a polynomial.
    poly_features = PolynomialFeatures(degree=d, include_bias=include_bias)
    # Instantiate a scaler.
    std_scaler = StandardScaler()
    # Instantiate a linear regressor.
    lin_reg = LinearRegression()

    # Create a pipeline of actions.
    polynomial_regression = Pipeline([
        ("poly_features", poly_features),
        ("std_scaler", std_scaler),
        ("lin_reg", lin_reg),
    ])

    lin_scores = cross_val_score(polynomial_regression, x_train, y_train, scoring='neg_mean_squared_error', cv=10)
    scores = np.sqrt(-lin_scores)
    mean_vec.append(scores.mean())
    std_vec.append(scores.std())
    
    # Perform polynomial regression.
    polynomial_regression.fit(x_train, y_train)
    
    y_train_predict = polynomial_regression.predict(x_train)
    
    y_val_predict = polynomial_regression.predict(x_val)    
    
    error_train.append(np.sqrt(mean_squared_error(y_train, y_train_predict)))

    error_val.append(np.sqrt(mean_squared_error(y_val, y_val_predict)))

# Plot results.
plt.figure(figsize=(20, 5))
ax = plt.subplot(1, 3, 1)
plt.plot(range(1, 13), mean_vec,  label='Erro quadrático médio')
plt.xticks(range(0, 13, 2))
plt.xlim([1, 12])
plt.legend()
plt.grid()

ax = plt.subplot(1, 3, 2)
plt.plot(range(1, 13), std_vec,  label='Desvio padrão do erro')
plt.xticks(range(0, 13, 2))
plt.xlim([1, 12])
plt.legend()
plt.grid()

ax = plt.subplot(1, 3, 3)
plt.plot(range(1, 13), error_train, label='Erro de trainamento')
plt.plot(range(1, 13), error_val, label='Erro de validação')
plt.xticks(range(0, 13, 2))
plt.xlim([1, 12])
plt.legend()
plt.grid()

#Show the plot.
plt.show()

In [ ]:
degrees = [1, 3, 4, 10]

plt.figure(figsize=(24, 5))
for i in range(len(degrees)):
    ax = plt.subplot(1, len(degrees), i + 1)

    # Instantiate a polynomial.
    poly_features = PolynomialFeatures(degree=degrees[i], include_bias=include_bias)
    # Instantiate a scaler.
    std_scaler = StandardScaler()
    # Instantiate a linear regressor.
    lin_reg = LinearRegression()

    # Create a pipeline of actions.
    polynomial_regression = Pipeline([
        ("poly_features", poly_features),
        ("std_scaler", std_scaler),
        ("lin_reg", lin_reg),
    ])

    error_training = []
    error_validation = []
    error_test = []
    for m in range(1,len(x_train)+1):

        # Perform polynomial regression.
        polynomial_regression.fit(x_train[:m], y_train[:m])

        # Use the trained model for prediction of the training set.
        y_train_predict = polynomial_regression.predict(x_train[:m])

        # Use the trained model for prediction of the validation set.
        y_val_predict = polynomial_regression.predict(x_val)       

        # Calculate MSE for training set.
        error_training.append(mean_squared_error(y_train[:m], y_train_predict))

        # Calculate MSE for validation set.
        error_validation.append(mean_squared_error(y_val, y_val_predict))     

    plt.plot(range(1,len(x_train)+1), np.sqrt(error_training), label='Conjunto de treinamento')
    plt.plot(range(1,len(x_train)+1), np.sqrt(error_validation), label='Conjunto de validação')
    plt.yscale('log')
    plt.xlabel('Tamanho do conjunto de treinamento', fontsize=14)
    plt.ylabel('Raiz Quadrada do Erro Quadrático Médio', fontsize=14)
    plt.grid()
    #plt.ylim([1e-2, 10])
    plt.title('Polinômio de ordem '+str(degrees[i]))
    plt.legend(loc='best')
    
#Show the plot.
plt.show()

## Model trainning

In [ ]:
degree = 3
include_bias = False

# Instantiate a polynomial with the given degree.
poly_features = PolynomialFeatures(degree=degree, include_bias=include_bias)
# Instantiate a scaler that will standardize the features.
std_scaler = StandardScaler()
# Instantiate a linear regressor.
lin_reg = LinearRegression()

# Create a pipeline of actions.
model = Pipeline([
    ("poly_features", poly_features),
    ("std_scaler", std_scaler),
    ("lin_reg", lin_reg),
])

# Train model.
model.fit(X, y)

# Perform prediction for 100 days.
X_pred = np.arange(1, 70).reshape(-1, 1)
y_pred = model.predict(X_pred)
X_pred = X_pred.reshape(1, -1)[0]

## The growth curve in the model and other countries

In [ ]:
fig = go.Figure(
    layout=go.Layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
    )
)

for country, df_country in df_filtered.groupby(by='country'):
    fig.add_trace(
        go.Scatter(
            x=df_country['days_from_k_case'],
            y=df_country['active'],
            mode='lines',
            name=country+' - dia do início: '+str(initial_day.date())
        )
    )

fig.add_trace(
    go.Scatter(
        x=X_pred,
        y=y_pred,
        mode='markers',
        name='Prediction'
    )
)


fig.add_trace(
    go.Scatter(
        x=X_pred,
        y=40600*np.ones(len(X_pred)),
        mode='lines',
        name='Número total de leitos de UTIs (adulto+pediátrico)'
    )
)


fig.add_trace(
    go.Scatter(
        x=X_pred,
        y=17900*np.ones(len(X_pred)),
        mode='lines',
        name='Número total de leitos de UTIs (SUS)'
    )
)

## When will Brazil reach the limit of the public health system (SUS)?

In [ ]:
dia_do_colapso_leitos_uti_total = min([i+1 for i,v in enumerate(y_pred) if v >= 40600])
dia1=df_filtered[(df_filtered['country']=='Brazil')]['date'].min() + timedelta(days=dia_do_colapso_leitos_uti_total)

dia_do_colapso_leitos_uti_sus = min([i+1 for i,v in enumerate(y_pred) if v >= 17900])
dia2=df_filtered[(df_filtered['country']=='Brazil')]['date'].min() + timedelta(days=dia_do_colapso_leitos_uti_sus)

print('dia_do_colapso_leitos_uti_total: ', dia1.date())
print('dia_do_colapso_leitos_uti_sus: ', dia2.date())

## Currently, how far Brazil and Italy are from the model?

In [ ]:
y_brazil = df_filtered[(df_filtered['country']=='Brazil')]['active']
brazil_model_diff = y_brazil ** (1/y_pred[0:len(y_brazil)])
brazil_model_mean_diff = np.mean(brazil_model_diff)
print('The difference between the model and the real data is %.2f %% in Brazil'%((brazil_model_mean_diff-1)*100))

## The growth curve in Brazil and Italy, and our distance for the max capacity in UTI (*Unidade de Tratamento Intensivo*, Intensive Care Unit) the Brazilian health system

**Note:** Values for "Max capacity of UTI in public health system (SUS)" and "Max capacity of UTI in public health system (SUS) + private sector" are based on Brazilian health system (SUS). Source: Estadão

In [ ]:
fig = go.Figure(
    layout=go.Layout(
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)'
    )
)

fig.add_trace(
    go.Scatter(
        x=X_pred,
        y=y_pred*brazil_model_mean_diff,
        mode='markers',
        name='Brazil Prediction (active cases)',
        marker={
            'color':'black',
            'size': 2
        }
    )
)

fig.add_trace(
    go.Scatter(
        x=X_pred,
        y=y_pred*0.1*brazil_model_mean_diff,
        mode='markers',
        name='Brazil Prediction (active cases with UTI need)',
        marker={
            'color':'orange',
            'size': 4
        }
    )
)

fig.add_trace(
    go.Scatter(
        x=df_filtered[df_filtered['country']=='Italy']['days_from_k_case'],
        y=df_filtered[df_filtered['country']=='Italy']['active'],
        mode='lines',
        name='Italy',
        line={
            'color':'red',
            'width':5
            
        }
    )
)

fig.add_trace(
    go.Scatter(
        x=df_filtered[df_filtered['country']=='Brazil']['days_from_k_case'],
        y=df_filtered[df_filtered['country']=='Brazil']['active'],
        mode='lines',
        name='Brazil',
        line={
            'color':'black',
            'width':5
            
        }
    )
)

fig.add_trace(
    go.Scatter(
        name="Max capacity of UTI in public health system (SUS)",
        x = [0, 100],
        y = [27400, 27400],
    )
)

fig.add_trace(
    go.Scatter(
        name="Max capacity of UTI in public health system (SUS) + private sector",
        x = [0, 100],
        y = [17900, 17900],
    )
)